In [ ]:
#Python standard library imports
import datetime as dt
import pathlib
import re
#Scientific Python ecosystem imports
import pandas as pd
#Text mining packages
import nltk as nltk
#from whoosh  import stem
#MM Import
import datetime as dt
import pathlib
import logging

from docx2python import docx2python

from polmap.polmap import preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
## 1.a) Read all files in input directory and select allowed filetypes

input_dir = pathlib.Path.cwd() / 'pdf_re' / 'Test' #MM let user provide an input dir
input_folder_name = input_dir.name

allowed_filetypes=['.pdf','.html','.mhtml','.doc','.docx']

files = sorted(input_dir.glob('**/*.*'))
files = [ file for file in files if file.suffix in allowed_filetypes]
#MM assert files==False and log assertion error.

In [ ]:
## 1.b) Create output folder structure based on input name, date and time of exectution

date = dt.datetime.now().date().isoformat() #def make_directories(project='TEI'): #MM start func definition
hour = dt.datetime.now().time().isoformat(timespec='seconds').replace(':', '')
current_date = '_'+date+'_T'+hour

project_title = input_folder_name+str(current_date) 

out_dir = pathlib.Path.cwd() / 'output' / project_title #Beginning of try block
log_dir = out_dir / 'logs'
results_dir = out_dir / 'results'
docs2txt_dir = out_dir / 'docs2txt'
stemmed_doctext_dir = out_dir / 'docs2txt_stemmed'

dir_dict = { directory: directory.mkdir(mode=0o777, parents=True, exist_ok=True) for directory in [out_dir, log_dir, results_dir, docs2txt_dir, ] } 

In [ ]:
PDFtext = []
counter = 0
for doc_path in files:
    counter += 1
    try:
        policy_text=[]
        doc_text = doc2text(doc_path)
        while '\n\n\n\n' in doc_text : doc_text = doc_text.replace('\n\n\n\n', '\n\n\n') #docx2python specific fix. would probably fit better elsewhere
        policy_text.append(doc_text)
        doctext_ = doc_path.parts[doc_path.parts.index(input_dir.name)+1:]
        doctext_name =  docs2txt_dir.joinpath(*doctext_)
        doctext_name.parent.mkdir(mode=0o777, parents=True, exist_ok=True)
        doctext_name = doctext_name.parent.joinpath(doctext_name.stem+'.txt')
        with open(doctext_name, 'w') as file_:
           file_.write(doc_text)
        PDFtext.append(['/'.join(doctext_),' ; '.join(policy_text)])
    except Exception as excptn: #MM I'd log errors as described in https://realpython.com/python-logging/, we need to test this.
        logging.exception('{doc_file} raised exception {exception} \n\n'.format(doc_file=doc_item.name, exception=excptn))


In [ ]:
print(PDFtext[0])